# googleMapReviewLabeling 

* 코드 목적: 수동으로 부여한 라벨링한 1개의 파일을 각 keyword(관광지 영문명)에 따라 파일을 형성해주는 코드 

## Target 설정
각 관광지별 데이터파일을 가지고 있는 상위 폴더명을 Target으로 설정한다. 

In [1]:
ls ./filtered/google_review

google_reviews_labeled_8429.csv             labeled_spellcheck_8420/
google_reviews_labeled_spacing_8421.csv     pred/
google_reviews_labeled_spellcheck_8420.csv  spacing_8421/
google_reviews_nolabel_8429.csv             spacing_spellcheck_8420/
google_reviews_spacing_8421.csv             spellcheck_8420/
labeled_spacing_8421/


In [2]:
target = 'spellcheck_8420' # spellcheck_8420(띄어쓰기를 적용한 8420개의 데이터를 가진 파일

## 라이브러리 설정

In [3]:
import pandas as pd
import os
from tqdm import tqdm

import warnings

warnings.filterwarnings(action='ignore')

## 경로 설정 

In [4]:
filtered_path = os.getcwd() + '/filtered/google_review/'
labeled_path = os.getcwd() + '/labeled/google_reviews_labeled.csv'

## 여러개의 데이터 파일을 불러오고 하나로 합치기

In [5]:
# english keyword to korean keyword
eng2kor = {
            'jangtae_mountain': '장태산',
             'gyejok_mountain': '계족산',
             'dongchundang': '동춘당',
             'uineungjeongi_street': '으느정이문화의거리',
             'ppuri_park': '뿌리공원',
             'expo_science_park': '엑스포',
             'sungsimdang_bakery': '성심당',
             'water_barrel': '수통골',
             'yuseong_hotspring': '유성온천',
             'hanbat_arboretum': '한밭수목원',
             'science_museum': '국립중앙과학관',
             'daecheong_lake': '대청호',
             'art_culture_complex': '대전문화예술단지',
             'observatory': '시민천문대',
             'oworld_zoo': '오월드'
}

# korean keyword to english keyword
kor2eng = {v:k for k, v in eng2kor.items()}

print('영어 keyword => 한국어 keyword')
print(eng2kor)
print()
print('한국어 keyword => 영어 keyword')
print(kor2eng)

영어 keyword => 한국어 keyword
{'jangtae_mountain': '장태산', 'gyejok_mountain': '계족산', 'dongchundang': '동춘당', 'uineungjeongi_street': '으느정이문화의거리', 'ppuri_park': '뿌리공원', 'expo_science_park': '엑스포', 'sungsimdang_bakery': '성심당', 'water_barrel': '수통골', 'yuseong_hotspring': '유성온천', 'hanbat_arboretum': '한밭수목원', 'science_museum': '국립중앙과학관', 'daecheong_lake': '대청호', 'art_culture_complex': '대전문화예술단지', 'observatory': '시민천문대', 'oworld_zoo': '오월드'}

한국어 keyword => 영어 keyword
{'장태산': 'jangtae_mountain', '계족산': 'gyejok_mountain', '동춘당': 'dongchundang', '으느정이문화의거리': 'uineungjeongi_street', '뿌리공원': 'ppuri_park', '엑스포': 'expo_science_park', '성심당': 'sungsimdang_bakery', '수통골': 'water_barrel', '유성온천': 'yuseong_hotspring', '한밭수목원': 'hanbat_arboretum', '국립중앙과학관': 'science_museum', '대청호': 'daecheong_lake', '대전문화예술단지': 'art_culture_complex', '시민천문대': 'observatory', '오월드': 'oworld_zoo'}


In [6]:
def concatCsv(data_type:str, keyword_kor:str):
    '''
    정제 전 폴더 data와 정제 후 폴더 filtered간의 파일 구성이 따르기 때문에 
    주어진 폴더명에 따라 dataframe을 합해주는 함수
    '''
    project_path = '/home/aiffel-dj19/jungcheck/DataPreprocessing'

    if data_type == 'data':
        keyword_path = data_type + '/google_review/' + keyword_kor
        folder_path = os.path.join(project_path, keyword_path)
        print(f'folder name: {folder_path}')
        print(f'..: {os.listdir(folder_path)}')
        
        data_list = []
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            print(f'file path: {file_path}')

            df = pd.read_csv(file_path, encoding='utf-8') 
            search = file.split('.')[0]  # 파일명에서 파일형식(.csv) 제외 후, 검색어 추출             
            df['search'] = search
            df['keyword'] = kor2eng[keyword_kor] 
            data_list.append(df)
            print(f'해당 파일의 데이터 개수: {len(df)}')
            print()
        df = pd.concat(data_list, axis=0)
        print('-'*40)
        
    elif data_type == 'filtered':
        folder_path = os.path.join(project_path, data_type + '/google_review/' + target)
        print(f'folder name: {folder_path}')
        
        data_list = []    
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            print(f'file path: {file_path}')

            df = pd.read_csv(file_path, encoding='utf-8')  # csv 파일 읽기
            print(f'해당 파일의 데이터 개수: {len(df)}')
            print()
            data_list.append(df)
            print('-'*40)
            
        df = pd.concat(data_list, axis=0)
    return df

In [7]:
print("==== filtered_df 불러오기 ====")
filtered_fds = os.listdir(filtered_path) # fds: 'folders'
print(f'{filtered_path}내의 파일 개수: {len(filtered_fds)}개') 
print(filtered_fds)
print()

==== filtered_df 불러오기 ====
/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/내의 파일 개수: 11개
['google_reviews_spacing_8421.csv', 'labeled_spacing_8421', 'spacing_8421', 'google_reviews_labeled_spellcheck_8420.csv', 'labeled_spellcheck_8420', 'spacing_spellcheck_8420', 'spellcheck_8420', 'google_reviews_nolabel_8429.csv', 'pred', 'google_reviews_labeled_spacing_8421.csv', 'google_reviews_labeled_8429.csv']



In [8]:
print("==== 폴더 안의 하위 파일리스트 불러오기 ====")
filtered_df = concatCsv('filtered', filtered_fds)
print()

==== 폴더 안의 하위 파일리스트 불러오기 ====
folder name: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/spellcheck_8420
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/spellcheck_8420/expo_science_park_spellcheck.csv
해당 파일의 데이터 개수: 783

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/spellcheck_8420/oworld_zoo_spellcheck.csv
해당 파일의 데이터 개수: 446

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/spellcheck_8420/daecheong_lake_spellcheck.csv
해당 파일의 데이터 개수: 491

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/spellcheck_8420/uineungjeongi_street_spellcheck.csv
해당 파일의 데이터 개수: 333

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/spellcheck_8420/sungsimdang_bakery_spel

## 데이터 Describe
* 데이터 개수 확인 
* null값 확인 
* 라벨 초기화    
해당 요소들을 기존 labeling이 진행된 데이터와 label값을 복사할 데이터 각각 확인해줍니다.

In [9]:
print("==== filtered_df 데이터 개수 재확인 ====")
for keyword in eng2kor.keys():
    print(keyword,'=>', filtered_df[filtered_df['keyword'] == keyword]['search'].unique(),': ', len(filtered_df[filtered_df['keyword'] == keyword]['search']))
# filtered_df['comment'][filtered_df['comment'].map(len) == 1].unique()
print()

==== filtered_df 데이터 개수 재확인 ====
jangtae_mountain => ['장태산자연휴양림메타세콰이어산림욕장' '장태산자연휴양림' '장태산자연휴양림전망대' '장태산자연휴양림숲속의집'] :  518
gyejok_mountain => ['계족산황톳길' '계족산(봉우리읍내동)' '계족산성'] :  483
dongchundang => ['회덕동춘당' '동춘당공원'] :  329
uineungjeongi_street => ['으능정이거리' '대전스카이로드' '으능정이문화거리'] :  333
ppuri_park => ['뿌리공원'] :  426
expo_science_park => ['세계엑스포기념품박물관' '엑스포음악분수' '엑스포과학공원' '엑스포다리' '한빛탑'] :  783
sungsimdang_bakery => ['성심당DCC점' '성심당대전역점' '성심당본점' '성심당케잌부띠끄본점' '성심당롯데백화점대전점'] :  1863
water_barrel => ['계룡산국립공원수통골지구' '수통골유원지'] :  685
yuseong_hotspring => ['유성온천공원' '유성온천족욕체험장'] :  431
hanbat_arboretum => ['한밭수목원'] :  402
science_museum => ['국립중앙과학관'] :  381
daecheong_lake => ['대청호' '대청댐물문화관' '대청호반자연생태공원'] :  491
art_culture_complex => ['대전예술의전당' '이응노미술관' '대전시립미술관'] :  823
observatory => ['대전시민천문대'] :  26
oworld_zoo => ['오월드'] :  446



In [10]:
print("==== filtered_df null값 확인 ====")
print(filtered_df.isnull().sum())
print()

print("==== filtered_df 라벨 -1로 초기화 ====")
filtered_df['label'] = -1 # -1이 아예 없는 값이므로 -1로 초기화 
print()

print("==== filtered_df 데이터 타입 변환 ====")
filtered_df['label'] = filtered_df['label'].apply(lambda row: int(row))
filtered_df['ratings'] = filtered_df['ratings'].apply(lambda row: int(row))
filtered_df['name'] = filtered_df['name'].apply(lambda row:row.strip(' '))
print()

==== filtered_df null값 확인 ====
name       0
ratings    0
date       0
comment    0
search     0
keyword    0
dtype: int64

==== filtered_df 라벨 -1로 초기화 ====

==== filtered_df 데이터 타입 변환 ====



In [11]:
print("==== labeled_df 불러오기 ====")
labeled_df = pd.read_csv(labeled_path)
print()

print("==== labeled_df null값 확인 ====")
print(labeled_df.name.isnull().sum())
print(labeled_df.keyword.isnull().sum())
print(labeled_df.comment.isnull().sum())
print()

print("==== labeled_df null값 제거 ====")
labeled_df = labeled_df.dropna(subset=['name', 'keyword', 'comment'])
print()

print("==== labeled_df null값 제거 확인 ====")
print(labeled_df.name.isnull().sum())
print(labeled_df.keyword.isnull().sum())
print(labeled_df.comment.isnull().sum())
print()

print("==== labeled_df 데이터 타입 변환 ====")
labeled_df['label'] = labeled_df['label'].apply(lambda row: int(row))
labeled_df['ratings'] = labeled_df['ratings'].apply(lambda row: int(row))
labeled_df['name'] = labeled_df['name'].apply(lambda row:row.strip(' '))

==== labeled_df 불러오기 ====

==== labeled_df null값 확인 ====
30
30
30

==== labeled_df null값 제거 ====

==== labeled_df null값 제거 확인 ====
0
0
0

==== labeled_df 데이터 타입 변환 ====


In [12]:
print('라벨링전(filtered_df): ', len(filtered_df))
print('라벨링후(labeled_df): ', len(labeled_df))
print()

라벨링전(filtered_df):  8420
라벨링후(labeled_df):  8377



## 데이터 일치여부 확인 및 Label값 복사
기존에 라벨링이 진행된 데이터 파일과, 읽어오는 파일간에 데이터가 키워드가 동일한지 확인해줍니다. 관광지 개수가 데이터 수집 개수와 모델링 결과에 따라 달라짐에 따라 keyword가 변경되기도 했었기 때문에 오류방지용으로 서로간에 키워드나, 데이터 일치여부에 따라 라벨값을 복사해줍니다. 

In [13]:
issamekeywords = sorted(labeled_df['keyword'].unique()) == sorted(filtered_df['keyword'].unique())
print('==== 서로 키워드가 동일한지 확인 ====', issamekeywords)
print()

for i in tqdm(range(len(labeled_df))):
    lb = labeled_df['label'].iloc[i]
    filtered_df['label'][(filtered_df['date'] == labeled_df['date'].iloc[i]) & 
                         (filtered_df['ratings'] == labeled_df['ratings'].iloc[i]) & 
                         (filtered_df['name'] == labeled_df['name'].iloc[i]) &
                         (filtered_df['keyword'] == labeled_df['keyword'].iloc[i]) == True] = lb
#     print(lb, end='')
#     print(labeled_df['comment'].iloc[i])
    
print('라벨링전(filtered_df): ', len(filtered_df))
print('라벨링후(labeled_df): ', len(labeled_df))
print()

==== 서로 키워드가 동일한지 확인 ==== True



100%|██████████| 8377/8377 [00:27<00:00, 300.28it/s]

라벨링전(filtered_df):  8420
라벨링후(labeled_df):  8377



## Labeling 복사한 데이터프레임 파일로 저장

In [14]:
print("==== 데이터프레임 한 파일로 저장 ====")
# 전체 합친 데이터프레임 한 파일로 저장
filtered_df.to_csv('./filtered/google_review/google_reviews_labeled_' + target + '.csv', index=False)

# keyword마다 파일 저장 
keywords = sorted(filtered_df['keyword'].unique())

==== 데이터프레임 한 파일로 저장 ====


## Labeling 안 된 데이터 개수 확인

In [15]:
def comparisonDataframe():
    cnt = 0 
    for i in tqdm(range(len(labeled_df))):
        lb = labeled_df['label'].iloc[i]
        filtered_df['label'][(filtered_df['date'] == labeled_df['date'].iloc[i]) & 
                             (filtered_df['ratings'] == labeled_df['ratings'].iloc[i]) & 
                             (filtered_df['name'] == labeled_df['name'].iloc[i]) &
                             (filtered_df['keyword'] == labeled_df['keyword'].iloc[i]) == True] = lb
        tmp = filtered_df['comment'][(filtered_df['date'] == labeled_df['date'].iloc[i]) & 
                             (filtered_df['ratings'] == labeled_df['ratings'].iloc[i]) & 
                             (filtered_df['name'] == labeled_df['name'].iloc[i]) &
                             (filtered_df['keyword'] == labeled_df['keyword'].iloc[i]) == True]
        print(tmp)
        print('--===========================--')
        cnt += 1
    print(cnt)

In [16]:
print("==== labeling 안된 데이터 개수 확인 ====")
nolabel_cnts = 0
for key in keywords:
    nolabel_cnt = filtered_df['label'][(filtered_df['label'] == -1) & (filtered_df['keyword'] == key)].sum()
    nolabel_cnts = nolabel_cnts + nolabel_cnt
    print(key,'에서 라벨링 안된 데이터 개수: ', nolabel_cnt)
    
    df = filtered_df[filtered_df['keyword'] == key]
#     df.to_csv(f'./filtered/google_review/google_reviews_{key}.csv', index=False)
print('전체 라벨링 안된 데이터 개수: ', nolabel_cnts)

==== labeling 안된 데이터 개수 확인 ====
art_culture_complex 에서 라벨링 안된 데이터 개수:  -6
daecheong_lake 에서 라벨링 안된 데이터 개수:  -19
dongchundang 에서 라벨링 안된 데이터 개수:  -1
expo_science_park 에서 라벨링 안된 데이터 개수:  -16
gyejok_mountain 에서 라벨링 안된 데이터 개수:  0
hanbat_arboretum 에서 라벨링 안된 데이터 개수:  0
jangtae_mountain 에서 라벨링 안된 데이터 개수:  0
observatory 에서 라벨링 안된 데이터 개수:  -5
oworld_zoo 에서 라벨링 안된 데이터 개수:  0
ppuri_park 에서 라벨링 안된 데이터 개수:  0
science_museum 에서 라벨링 안된 데이터 개수:  -25
sungsimdang_bakery 에서 라벨링 안된 데이터 개수:  0
uineungjeongi_street 에서 라벨링 안된 데이터 개수:  0
water_barrel 에서 라벨링 안된 데이터 개수:  0
yuseong_hotspring 에서 라벨링 안된 데이터 개수:  0
전체 라벨링 안된 데이터 개수:  -72
